# ---------------------------------------构建用于训练的打包数据包-------------------------------

- ### 处理好图片和词表后开始将数据打包用于训练

In [2]:
import torch 
import torchvision.transforms as transforms
import torch.utils.data as data
import os
import pickle
import numpy as np
import nltk
from PIL import Image
from pycocotools.coco import COCO
import argparse
from torchvision import transforms

# import Ipynb_importer
import import_ipynb
from Bulid_Vocab_2 import Vocabulary

In [15]:
"""创建coco自定义数据类"""
class CocoDataset(data.Dataset):
    def __init__(self, root, json, vocab, transform=None):
        # root: 图像文件的根目录
        # json: COCO标注文件路径
        # vocab: 词汇表包装器
        # transform: 图像增广处理
        self.root = root
        self.coco = COCO(json)
        self.ids = list(self.coco.anns.keys()) # 获取coco数据集中所有标注的唯一标识符anns并转化为列表
        self.vocab = vocab
        self.transform = transform
        
    def __getitem__(self, index): # 类方法，用于根据索引获取值
        coco = self.coco
        vocab = self.vocab
        ann_id = self.ids[index] # 根据索引获取相应的标注的唯一识别符
        
        caption = coco.anns[ann_id]['caption'] # 获取标注的文件，此处是针对图片的描述
        img_id = coco.anns[ann_id]['image_id'] # 获取关联图像的唯一标识符
        path = coco.loadImgs(img_id)[0]['file_name'] # 通过图像id获取对应图像的文件名
        
        image = Image.open(os.path.join(self.root, path)).convert('RGB')# 图像路径有root和path图片名组成，打开图像
        
        if self.transform is not None: # 判断是否需要对图片进行增广处理
            image = self.transform(image)
        
        tokens = nltk.tokenize.word_tokenize(str(caption).lower()) # 对标注文本进行分词，并转换为小写
        caption = [] # 创建空字典
        caption.append(vocab('<start>')) # 在标注开头添加start起始符对应的序号
        caption.extend([vocab(token) for token in tokens]) # 遍历标注，将其转换为索引后加入到caption中
        caption.append(vocab('<end>')) # 添加end结束符对应的序号
        target = torch.Tensor(caption) # 将标注文件转化为Tensor
        
        return image, target  # 返回图片及其对应的处理好的标准文件
    
    def __len__(self):
        return len(self.ids) # 返回ids长度
# 返回的是一个元组的列表，元组包括图片及其对应的处理好的标注文件，

In [18]:
"""创建用于训练的batch所需要的格式"""
def collate_fn(data):
    """从图像和标注的元组列表中创建小批量张量。

我们应该构建自定义的 `collate_fn`，而不是使用默认的 `collate_fn`，因为默认情况下不支持合并标注（包括填充）。

参数:
    data: 元组列表（图像, 标注）。
        - 图像: 形状为 (3, 256, 256) 的 PyTorch 张量。
        - 标注: 形状为 (?) 的 PyTorch 张量，可变长度。

返回:
    images: 形状为 (batch_size, 3, 256, 256) 的 PyTorch 张量。
    targets: 形状为 (batch_size, padded_length) 的 PyTorch 张量。
    lengths: 列表，每个填充标注的有效长度。
"""
    data.sort(key=lambda x: len(x[1]), reverse=True) # 按data中【1】标注的长度进行降序排列
    images, captions = zip(*data) # 将排序后的data拆分成两个元组，images,captions

    images = torch.stack(images, 0) # 将图像元组合并成一个张量，就是所有图片叠加在一起

    lengths = [len(cap) for cap in captions] # 计算每个标注的长度
    targets = torch.zeros(len(captions), max(lengths)).long() # 创建一个全零张量，长度为标注的数量，宽度为最大标注的长度
    for i, cap in enumerate(captions): # 此处是将每个标注填充到相同的长度
        end = lengths[i]
        targets[i, :end] = cap[:end]
    return images, targets, lengths

# 将图片合成一个batch_size, 同时对应的标注文件整合

In [5]:
"""构建加载数据集"""
def get_loader(root, json, vocab, transform, batch_size, shuffle, num_workers):
    # 这一步是实例化自定义数据集类，返回包含图片及其对应标注的元组的列表
    coco = CocoDataset(root=root, json=json, vocab=vocab, transform=transform)
    
    # 将数据集打包
    data_loader = torch.utils.data.DataLoader(dataset=coco,
                                             batch_size=batch_size,
                                             shuffle=shuffle,
                                             num_workers=num_workers,
                                             collate_fn=collate_fn) # 自定义的collate_fn的方法
    return data_loader

# ----------------------------------------------测试能否正常运行------------------------------------

In [6]:
"""设置参数"""
parser = argparse.ArgumentParser() 

# 给定一些基础参数，包括文件路径等
parser.add_argument('--model_path', type=str, default='models/', help='保存训练模型的地方')
parser.add_argument('--crop_size', type=int, default=224, help='随机裁剪图片的大小')
parser.add_argument('--vocab_path', type=str, default='data/vocab.pkl', help='之前生成词典的路径')
parser.add_argument('--image_dir', type=str, default='data/resized2014', help='已经处理好大小的训练图片的路径')
parser.add_argument('--caption_path', type=str, default='data/annotations/captions_train2014.json',help='训练集标签的路径')
parser.add_argument('--log_step', type=int, default=10, help='打印训练进度的设定值')
parser.add_argument('--save_step', type=int, default=1000, help='保存模型节点的设定值')

# 设定模型的参数值
parser.add_argument('--embed_size', type=int, default=256, help='词嵌入向量的维度，也就是用多少维来表示一个词元')
parser.add_argument('--hidden_size', type=int, default=512, help='隐藏状态的维度')
parser.add_argument('--num_layers', type=int, default=1, help='LSTM层的数量')

# 设定训练的参数
parser.add_argument('--num_epochs', type=int, default=1, help='epoch数')
parser.add_argument('--batch_size', type=int, default=128, help='批量大小')
parser.add_argument('--num_workers', type=int, default=None, help='并行运算大小')
parser.add_argument('--learning_rate', type=float, default=0.001, help='学习率')

# 解析传入的命令行参数
args = parser.parse_args(args=[])
print(args)

Namespace(model_path='models/', crop_size=224, vocab_path='data/vocab.pkl', image_dir='data/resized2014', caption_path='data/annotations/captions_train2014.json', log_step=10, save_step=1000, embed_size=256, hidden_size=512, num_layers=1, num_epochs=1, batch_size=128, num_workers=None, learning_rate=0.001)


In [7]:
"""定义增广操作"""
transform = transforms.Compose([
    transforms.RandomCrop(args.crop_size), # 随即裁剪到指定大小
    transforms.RandomHorizontalFlip(), # 随机水平翻转
    transforms.ToTensor(), # 转成tensor
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))  # 标准化 
])

In [8]:
"""指定 vocab.pkl词表 文件的路径"""
file_path = 'data/vocab.pkl'

#使用 pickle 模块打开文件
with open(file_path, 'rb') as file:
     vocab = pickle.load(file)

In [19]:
"""测试能否正常生成批量数据"""
data_loader = get_loader(args.image_dir, args.caption_path, vocab, transform, args.batch_size, 
                              shuffle=True, num_workers=0) #num_workers=args.num_workers

loading annotations into memory...
Done (t=0.49s)
creating index...
index created!


In [20]:
images, targets, lengths = next(iter(data_loader))

In [22]:
print(images.shape, targets.shape, len(lengths))
# images: 128批量，3通道，224*224图像大小
# targets: 标注文件，128批量，统一填充成28个词元表示，每行中，每一个数字代表词元在字典中的索引
# lengths: list，每个数字表示，当前行真实的标注是多少个，其它的都是填充pad

torch.Size([128, 3, 224, 224]) torch.Size([128, 25]) 128


In [23]:
images.shape

torch.Size([128, 3, 224, 224])

In [24]:
print(targets)

tensor([[   1,  146,  364,  ..., 1997,   19,    2],
        [   1,    4,  578,  ..., 2776,   19,    2],
        [   1,    4, 2361,  ..., 4185,   19,    2],
        ...,
        [   1,  251,   53,  ...,    0,    0,    0],
        [   1,  252,  108,  ...,    0,    0,    0],
        [   1,    4,  116,  ...,    0,    0,    0]])


In [21]:
targets.shape

torch.Size([128, 26])

In [29]:
targets[0,]

tensor([   1,    4,  860, 8422,   55,   33, 1929,    7,    4,   31, 1063, 1073,
         968,    3,   87,  328,    3,    3, 7413, 8659,   87,    3,    3, 1075,
          19,    2])

In [22]:
lengths

[26,
 20,
 18,
 18,
 16,
 16,
 16,
 16,
 16,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 9]

In [24]:
type(lengths)

list

In [25]:
len(lengths)

128